In [1]:
import pandas as pd
import gc
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from category_encoders import HashingEncoder
#from xgboost import XGBClassifier
from catboost import CatBoostClassifier

PROPORTION = 1/10

# Print all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)


ctr_15 = pd.read_csv ("./datos/ctr_15.csv")
ctr_16 = pd.read_csv ("./datos/ctr_16.csv")
ctr_17 = pd.read_csv ("./datos/ctr_17.csv")
ctr_18 = pd.read_csv ("./datos/ctr_18.csv")
ctr_19 = pd.read_csv ("./datos/ctr_19.csv")
ctr_20 = pd.read_csv ("./datos/ctr_20.csv")
ctr_21 = pd.read_csv ("./datos/ctr_21.csv")
ctr_test = pd.read_csv ("./datos/ctr_test.csv")

print("Datos cargados")

# Load the train data
train_data = pd.concat([ctr_15,ctr_16,ctr_17,ctr_18,ctr_19,ctr_20])
test_data = ctr_21
# Load the test data
#eval_data = ctr_test

# separo label 1 y 0 (ya que hay muy pocos 1 y me quiero quedar con todfos)
train_data_1 = train_data[train_data["Label"] == 1]
train_data_0 = train_data[train_data["Label"] == 0]
#train_data_1 = pd.concat([train_data_1 for i in range(10)]) # me quedo con el 1000% de los datos con label 1

# me quedo con el 10% de los datos con label 0
train_data_0_sampled = train_data_0.sample(frac=PROPORTION, random_state=123)

# junto los datos
train_data = pd.concat([train_data_1, train_data_0_sampled])

# separo x e y
y_train = train_data["Label"]
X_train = train_data.drop(columns=["Label"])
## aca se me ocurrio la inteligente idea de separar la columna de tiempo en dia, hora y dia de la semana
X_train["auction_time"] = pd.to_datetime(X_train["auction_time"], unit='s')
X_train["action_day"] = X_train["auction_time"].dt.day
X_train["action_time"] = X_train["auction_time"].dt.hour
X_train["action_weekday"] = X_train["auction_time"].dt.weekday
# ahora borro la columna auction_time
X_train = X_train.drop(columns=["auction_time"]) 

# hago lo mismo con el test
y_test = test_data["Label"]
X_test = test_data.drop(columns=["Label"])
X_test["auction_time"] = pd.to_datetime(X_test["auction_time"], unit='s')
X_test["action_day"] = X_test["auction_time"].dt.day
X_test["action_time"] = X_test["auction_time"].dt.hour
X_test["action_weekday"] = X_test["auction_time"].dt.weekday
# ahora borro la columna auction_time
X_test = X_test.drop(columns=["auction_time"])

print("variables temporales creadas")

# listas, primero las voy a dropear para ver que lo otro funciona
# despues veo que hago con las fukin listas (me tienen arto)
#X_train = X_train.drop(columns=["action_list_1", "action_list_2"])
#X_test = X_test.drop(columns=["action_list_1", "action_list_2"])
# agrego una columna con la cantidad de elementos en la lista

# imputo los valores faltantes de las variables categoricas
categorical_columns = X_train.select_dtypes(exclude=["number"]).columns
categorical_columns_test = X_test.select_dtypes(exclude=["number"]).columns
#X_train[categorical_columns] = X_train[categorical_columns].fillna("missing")
#X_test[categorical_columns] = X_test[categorical_columns].fillna("missing")

print("valores faltantes imputados")

# aplicar label encoding a las variables categoricas porque el xgboost es exquisito
# label econder es una pija
# hacemos hasshin encoding. DE BAJA TAMBIEN
# voy a poner todo como str y usar catboost
X_train[categorical_columns] = X_train[categorical_columns].astype(str)
X_test[categorical_columns] = X_test[categorical_columns_test].astype(str)



del train_data, train_data_1, train_data_0, train_data_0_sampled, test_data, ctr_15, ctr_16, ctr_17, ctr_18, ctr_19, ctr_20, ctr_21
gc.collect()



Datos cargados
variables temporales creadas
valores faltantes imputados


23

Ahora entrenamos

In [2]:
# catboost para maximizar el auc-roc
#model = CatBoostClassifier(iterations=1000, depth=8, learning_rate=0.1, loss_function='Logloss', verbose=True, eval_metric='AUC', class_weights=[100, 1], early_stopping_rounds=100)
#model.fit(X_train, y_train, cat_features=categorical_columns.to_list())
##y_pred = model.predict(X_test)
#y_pred_proba = model.predict_proba(X_test)[:,1]
#y_pred = model.predict(X_test)
#print("roc auc score exacta: ", roc_auc_score(y_test, y_pred))
#print("roc auc score proba: ", roc_auc_score(y_test, y_pred_proba))
# con log loss roc auc score:  0.7794631113149908
# con predictproba roc auc score: 0.7846885451761818
# agregando eval_metric='AUC', early_stopping_rounds=100, class_weights=[1, 100] roc auc score:  roc auc score:  0.8563
# agregando eval_metric='AUC', early_stopping_rounds=100, class_weights=[100, 1] roc auc score: 0.8664
# agregando eval_metric='AUC', early_stopping_rounds=100, class_weights=[1000, 1] roc auc score: 0.8599
# agregando eval_metric='AUC', early_stopping_rounds=100, class_weights=[1, 1000] roc auc score: 0.8273
# agregando eval_metric='AUC', early_stopping_rounds=100, class_weights=[1, 100] roc auc score: 0.8563

# best loss functions for AUC-ROC are: 
loss_functions = ['Logloss', 'CrossEntropy']
learning_rates = [0.1, 0.01, 0.001]
depths = [8]
iterations = [500]
for lf in loss_functions:
    for lr in learning_rates:
        for d in depths:
            for i in iterations:
                model = CatBoostClassifier(iterations=i, depth=d, learning_rate=lr, loss_function=lf, verbose=False, eval_metric='AUC', early_stopping_rounds=100, class_weights=[100, 1])
                model.fit(X_train, y_train, cat_features=categorical_columns.to_list())
                y_pred_proba = model.predict_proba(X_test)[:,1]
                y_pred = model.predict(X_test)
                print("loss function: ", lf, "learning rate: ", lr, "depth: ", d, "iterations: ", i, "roc auc score proba: ", roc_auc_score(y_test, y_pred_proba))
                print("loss function: ", lf, "learning rate: ", lr, "depth: ", d, "iterations: ", i, "roc auc score exacta: ", roc_auc_score(y_test, y_pred))


loss function:  Logloss learning rate:  0.1 depth:  8 iterations:  500 roc auc score proba:  0.8594368537347682
loss function:  Logloss learning rate:  0.1 depth:  8 iterations:  500 roc auc score exacta:  0.5030151752403266
loss function:  Logloss learning rate:  0.01 depth:  8 iterations:  500 roc auc score proba:  0.8412389219551093
loss function:  Logloss learning rate:  0.01 depth:  8 iterations:  500 roc auc score exacta:  0.5
loss function:  Logloss learning rate:  0.001 depth:  8 iterations:  500 roc auc score proba:  0.7811007563611843
loss function:  Logloss learning rate:  0.001 depth:  8 iterations:  500 roc auc score exacta:  0.5


CatBoostError: catboost/private/libs/options/catboost_options.cpp:612: class weights takes effect only with Logloss, MultiClass, MultiClassOneVsAll and user-defined loss functions

In [3]:
## subset selection
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# Separar el conjunto de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=123)

# Modelo base de CatBoost
cat_model = CatBoostClassifier(iterations=100, depth=8, learning_rate=0.1, loss_function='Logloss', verbose=False, cat_features=categorical_columns)

# Forward selection
sfs_forward = SFS(cat_model, 
            k_features=10,
            forward=True,
            floating=False,
            scoring='roc_auc',
            cv=3)

sfs_forward = sfs_forward.fit(X_train, y_train)
X_train_forward = X_train[list(sfs_forward.k_feature_names_)]
X_val_forward = X_val[list(sfs_forward.k_feature_names_)]

# Modelo con forward selection
model_forward = CatBoostClassifier(iterations=100, depth=8, learning_rate=0.1, loss_function='Logloss', verbose=False, cat_features=categorical_columns)
model_forward.fit(X_train_forward, y_train)
y_preds_forward = model_forward.predict(X_val_forward)
print("AUC CatBoost forward: ", roc_auc_score(y_val, y_preds_forward))

# Backward selection
sfs_backward = SFS(cat_model,
            k_features=10,
            forward=False,
            floating=False,
            scoring='roc_auc',
            cv=3)

sfs_backward = sfs_backward.fit(X_train, y_train)
X_train_backward = X_train[list(sfs_backward.k_feature_names_)]
X_val_backward = X_val[list(sfs_backward.k_feature_names_)]

# Modelo con backward selection
model_backward = CatBoostClassifier(iterations=100, depth=8, learning_rate=0.1, loss_function='Logloss', verbose=False, cat_features=categorical_columns)
model_backward.fit(X_train_backward, y_train)
y_preds_backward = model_backward.predict(X_val_backward)
print("AUC CatBoost backward: ", roc_auc_score(y_val, y_preds_backward))

# SelectFromModel
model_select = CatBoostClassifier(iterations=100, depth=8, learning_rate=0.1, loss_function='Logloss', verbose=False, cat_features=categorical_columns)
model_select.fit(X_train, y_train)

selector = SelectFromModel(model_select, prefit=True)
X_train_selected = selector.transform(X_train)
X_val_selected = selector.transform(X_val)

model_selected = CatBoostClassifier(iterations=100, depth=8, learning_rate=0.1, loss_function='Logloss', verbose=False, cat_features=categorical_columns)
model_selected.fit(X_train_selected, y_train)

y_preds_selected = model_selected.predict(X_val_selected)
print("AUC CatBoost con SelectFromModel: ", roc_auc_score(y_val, y_preds_selected))

# Comparación de resultados
print("Resultados:")
print("AUC CatBoost forward: ", roc_auc_score(y_val, y_preds_forward))
print("AUC CatBoost backward: ", roc_auc_score(y_val, y_preds_backward))
print("AUC CatBoost con SelectFromModel: ", roc_auc_score(y_val, y_preds_selected))




c:\Users\angel\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(


ValueError: 
All the 3 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\angel\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\angel\AppData\Local\Programs\Python\Python312\Lib\site-packages\catboost\core.py", line 5245, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, graph, sample_weight, None, None, None, None, baseline, use_best_model,
  File "c:\Users\angel\AppData\Local\Programs\Python\Python312\Lib\site-packages\catboost\core.py", line 2395, in _fit
    train_params = self._prepare_train_params(
                  

In [ ]:
### esto no va



X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=123)

sfs_forward = SFS(xgb_model, 
            k_features=10,
            forward=True,
            floating=False,
            scoring='roc_auc',
            cv=3)

sfs_forward = sfs_forward.fit(X_train, y_train)
X_train_forward = X_train[list(sfs_forward.k_feature_names_)]
X_val_forward = X_val[list(sfs_forward.k_feature_names_)]

model_forward = XGBClassifier(n_estimators=10, learning_rate=0.1, max_depth=7, random_state=123, objective="binary:logistic", eval_metric="auc", use_label_encoder=False)
model_forward.fit(X_train_forward, y_train)
y_preds_forward = model_forward.predict(X_val_forward)
print("AUC XGBoost forward: ", roc_auc_score(y_val, y_preds_forward))

sfs_backward = SFS(xgb_model,
            k_features=10,
            forward=False,
            floating=False,
            scoring='roc_auc',
            cv=3)

sfs_backward = sfs_backward.fit(X_train, y_train)
X_train_backward = X_train[list(sfs_backward.k_feature_names_)]
X_val_backward = X_val[list(sfs_backward.k_feature_names_)]

model_backward = XGBClassifier(n_estimators=10, learning_rate=0.1, max_depth=7, random_state=123, objective="binary:logistic", eval_metric="auc", use_label_encoder=False)
model_backward.fit(X_train_backward, y_train)
y_preds_backward = model_backward.predict(X_val_backward)
print("AUC XGBoost backward: ", roc_auc_score(y_val, y_preds_backward))


model_select = XGBClassifier(n_estimators=10, learning_rate=0.1, max_depth=7, random_state=123, objective="binary:logistic", eval_metric="auc", use_label_encoder=False)
model_select.fit(X_train, y_train)

selector = SelectFromModel(model_select, prefit=True)
X_train_selected = selector.transform(X_train)
X_val_selected = selector.transform(X_val)

model_selected = XGBClassifier(n_estimators=10, learning_rate=0.1, max_depth=7, random_state=123, objective="binary:logistic", eval_metric="auc", use_label_encoder=False)
model_selected.fit(X_train_selected, y_train)

y_preds_selected = model_selected.predict(X_val_selected)
print("AUC XGBoost con SelectFromModel: ", roc_auc_score(y_val, y_preds_selected))

# comparamos
print("resultados:")
print("AUC XGBoost forward: ", roc_auc_score(y_val, y_preds_forward))
print("AUC XGBoost backward: ", roc_auc_score(y_val, y_preds_backward))
print("AUC XGBoost con SelectFromModel: ", roc_auc_score(y_val, y_preds_selected))

print("chequear cual es mejor que el de arriba")


